In [38]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import logging

from sklearn.preprocessing import StandardScaler

from scipy.stats import skew, kurtosis

from utils.general import *

import json

import warnings

from sklearn.feature_selection import mutual_info_classif, mutual_info_regression

In [39]:
warnings.filterwarnings('ignore')

In [40]:
random_state = 42
rng = np.random.default_rng(random_state)

# Data Loading And Preprocessing

In [41]:
billboard_df = pd.read_csv("../data/billboard/hot-100_all.csv")
spotify_df = pd.read_csv("../data/spofity/songs.csv")
audio_analysis_df = pd.read_csv("../data/audio/audio_features_full.csv")

In [42]:
billboard_df.drop(axis=1, inplace=True, labels=['image', 'artist'])
billboard_df['date'] = pd.to_datetime(billboard_df['date'])
spotify_df.drop(axis=1, inplace=True, labels=['spotify_name',
                                              'artist',
                                              'artist_genres',
                                              'spotify_id',
                                              'spotify_id',
                                              'spotify_uri',
                                              'spotify_external_url',
                                              'spotify_artist_popularity',
                                              'preview_url',
                                              'preview_url_audio',
                                              'full_audio',
                                              'full_audio_duration_s'
                                              ])
audio_analysis_df.drop(axis=1, inplace=True, labels=['name', 'tempo'])

In [43]:
songs_df = spotify_df.merge(audio_analysis_df, how='inner', on='billboard_name')
songs_df

,billboard_name,duration_ms,spotify_popularity,spotify_artist_popularity_mean,explicit,danceability,energy,key,loudness,mode,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
0,All I Want For Christmas Is You,241106.0,69.0,81.000000,False,0.336,0.627,7.0,-7.463,1.0,...,1.619618,99.735660,-3.865835,99.024666,2.729783,112.219580,-7.488522,122.569650,2.323859,141.572560
1,Rockin' Around The Christmas Tree,126266.0,62.0,59.000000,False,0.589,0.472,8.0,-8.749,1.0,...,-1.039626,78.420586,-4.437555,55.536427,3.890496,70.359543,0.014326,77.899239,6.889563,93.610161
2,Jingle Bell Rock,130973.0,62.0,50.000000,False,0.754,0.424,2.0,-8.463,1.0,...,1.430321,58.685158,-4.030815,67.332291,1.802275,58.469532,-5.335912,53.423290,0.133941,58.774597
3,A Holly Jolly Christmas,135533.0,54.0,48.000000,False,0.683,0.375,0.0,-13.056,1.0,...,-1.355817,60.197350,-6.695084,52.782772,-4.325858,66.221947,-3.533713,50.849602,-1.266797,90.991325
4,Circles,215280.0,86.0,91.000000,False,0.695,0.762,0.0,-3.497,1.0,...,2.563944,78.141319,-12.359889,83.661438,4.207565,65.643173,-5.280680,54.441185,-0.751733,59.799530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,Christmas Isn't Canceled (Just You),231549.0,41.0,77.000000,False,0.580,0.789,1.0,-4.918,0.0,...,5.321839,70.412506,1.213545,77.698616,4.693950,85.284431,-2.604682,76.687698,5.805956,77.433144
1419,Moved To Miami,222225.0,66.0,88.500000,True,0.717,0.444,1.0,-11.126,1.0,...,4.598643,180.801086,-4.373017,110.878738,4.545245,111.550697,4.918246,63.780304,8.114554,75.806396
1420,Hibachi,170413.0,69.0,86.333333,True,0.681,0.522,5.0,-8.740,0.0,...,8.091636,79.854568,-1.646704,123.382797,5.309008,98.507568,0.213412,80.767159,2.804790,73.490234
1421,Thailand,200958.0,70.0,84.000000,True,0.875,0.478,7.0,-10.562,1.0,...,10.420262,91.743813,-2.071233,75.112267,8.341298,97.730263,-0.260812,69.875168,1.959964,62.722679


In [44]:
# print(songs_df.apply(lambda x: x.nunique()))
songs_df.describe()

,duration_ms,spotify_popularity,spotify_artist_popularity_mean,danceability,energy,key,loudness,mode,speechiness,acousticness,...,mfcc16_mean,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var
count,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,...,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000,1423.000000
mean,200696.557976,67.404076,83.302977,0.666045,0.622016,5.153197,-6.747289,0.622628,0.139837,0.223816,...,2.398114,84.513799,-3.647905,82.115186,2.668189,80.818220,-2.539943,79.075432,1.677588,80.371770
std,50770.975125,17.661345,12.018877,0.151579,0.162841,3.600340,2.579960,0.484900,0.125819,0.253148,...,3.631675,24.471115,3.398574,24.637780,3.253561,24.665541,3.023733,24.767819,3.186344,26.740148
min,32000.000000,0.000000,0.000000,0.150000,0.007600,0.000000,-33.663000,0.000000,0.023200,0.000003,...,-13.240079,32.670311,-18.392536,31.161884,-10.105947,31.325922,-14.229393,28.560005,-13.992401,28.597084
25%,170322.000000,64.000000,78.000000,0.570000,0.525000,1.000000,-7.841500,0.000000,0.042800,0.031550,...,0.246056,68.009289,-5.805181,65.208897,0.716595,64.160015,-4.468847,62.009989,-0.239844,61.936447
50%,195428.000000,70.000000,86.000000,0.680000,0.633000,5.000000,-6.360000,1.000000,0.079800,0.121000,...,2.442511,81.295982,-3.595455,78.402687,2.753472,77.774918,-2.510375,75.437561,1.633557,75.101379
75%,223599.000000,77.000000,91.000000,0.776000,0.733500,8.000000,-5.077500,1.000000,0.218000,0.321500,...,4.722763,98.500587,-1.416037,94.838615,4.810408,93.118820,-0.625247,91.067223,3.652482,93.510078
max,613026.000000,95.000000,100.000000,0.965000,0.984000,11.000000,-1.321000,1.000000,0.699000,0.995000,...,13.629806,207.025589,6.795540,219.371109,12.521308,229.869766,7.686097,229.040588,14.772246,226.710175


In [45]:
songs_df.isna().sum()

billboard_name                    0
duration_ms                       0
spotify_popularity                0
spotify_artist_popularity_mean    0
explicit                          0
                                 ..
mfcc18_var                        0
mfcc19_mean                       0
mfcc19_var                        0
mfcc20_mean                       0
mfcc20_var                        0
Length: 75, dtype: int64

In [46]:
songs_df = pd.get_dummies(songs_df, prefix=['explicit'], columns=['explicit'])

# Feature Engineering

### I don't know what made me think we need *MORE* features, but here's more features...
Some extra feature engineering for spotify's audio analysis object that did't happen during dataset construction.

In [47]:
# this code is so slow it hurts me...

def spotify_audio_analysis(filename):
    file = open(filename)
    analysis_json = json.load(file)
    file.close()
    timbre_coefs = np.arange(13, 1, -1)
    bars = analysis_json['bars']
    beats = analysis_json['beats']
    sections = analysis_json['sections']
    segments = analysis_json['segments']
    tatums = analysis_json['tatums']

    return pd.DataFrame(data={
        # "time_signature": [analysis_json['track']['time_signature']],
        # "mode": [analysis_json['track']['mode']],
        "num_bars": len(bars),
        "mean_bar_duration": np.mean([b['duration'] for b in bars]),
        "var_bar_duration": np.var([b['duration'] for b in bars]),
        "num_beats": len(beats),
        "mean_beat_duration": np.mean([b['duration'] for b in beats]),
        "var_beat_duration": np.var([b['duration'] for b in beats]),
        "num_sections": len(sections),
        "mean_section_duration": np.mean([s['duration'] for s in sections]),
        "var_section_duration": np.var([s['duration'] for s in sections]),
        "mean_section_tempo": np.mean([s['tempo'] for s in sections]),
        "var_section_tempo": np.var([s['tempo'] for s in sections]),
        "mean_section_loudness": np.mean([s['loudness'] for s in sections]),
        "var_section_loudness": np.var([s['loudness'] for s in sections]),
        # other section stuff
        "num_segments": len(segments),
        "mean_segment_duration": np.mean([s['duration'] for s in segments]),
        "var_segment_duration": np.var([s['duration'] for s in segments]),
        "mean_segment_loudness_start": np.mean([s['loudness_start'] for s in segments]),
        "var_segment_loudness_start": np.var([s['loudness_start'] for s in segments]),
        "mean_segment_loudness_max": np.mean([s['loudness_max'] for s in segments]),
        "var_segment_loudness_max": np.var([s['loudness_max'] for s in segments]),
        "mean_segment_loudness_max_time": np.mean([s['loudness_max_time'] for s in segments]),
        "var_segment_loudness_max_time": np.var([s['loudness_max_time'] for s in segments]),
        "mean_segment_loudness_end": np.mean([s['loudness_end'] for s in segments]),
        "var_segment_loudness_end": np.var([s['loudness_end'] for s in segments]),
        "mean_segment_loudness_start_max_diff": np.mean([abs(s['loudness_start'] - s['loudness_max']) for s in segments]),
        "var_segment_loudness_start_max_diff": np.var([abs(s['loudness_start'] - s['loudness_max']) for s in segments]),
        "mean_segment_loudness_start_end_diff": np.mean([abs(s['loudness_start'] - s['loudness_end']) for s in segments]),
        "var_segment_loudness_start_end_diff": np.var([abs(s['loudness_start'] - s['loudness_end']) for s in segments]),
        "mean_segment_loudness_max_end_diff": np.mean([abs(s['loudness_max'] - s['loudness_end']) for s in segments]),
        "var_segment_loudness_max_end_diff": np.var([abs(s['loudness_max'] - s['loudness_end']) for s in segments]),
        "mean_segment_num_pitches": np.mean([len(s['pitches']) for s in segments]),
        "var_segment_num_pitches": np.var([len(s['pitches']) for s in segments]),
        "mean_segment_num_pure_pitches": np.mean([len(np.array(s['pitches'])[np.array(s['pitches']) > 0.5]) for s in segments]),
        "var_segment_num_pure_pitches": np.var([len(np.array(s['pitches'])[np.array(s['pitches']) > 0.5]) for s in segments]),
        "mean_segment_timbre": np.mean([np.dot(s['timbre'], timbre_coefs) for s in segments]),
        "var_segment_timbre": np.var([np.dot(s['timbre'], timbre_coefs) for s in segments]),
        "num_tatums": len(tatums),
        "mean_tatum_duration": np.mean([t['duration'] for t in tatums]),
        "var_tatum_duration": np.var([t['duration'] for t in tatums]),
    }, index=[0])

##### This will take a little bit to run

In [48]:
aa = pd.DataFrame()
for i in range(len(songs_df)):
    aa = pd.concat([aa, spotify_audio_analysis("../data/spofity/audio_analysis/" + spotify_df['audio_analysis_file'].iloc[0])], axis=0)
aa.reset_index(inplace=True)
songs_df = pd.concat([songs_df, aa], axis=1)
songs_df.drop(labels='index', axis=1, inplace=True)

In [49]:
songs_df

,billboard_name,duration_ms,spotify_popularity,spotify_artist_popularity_mean,danceability,energy,key,loudness,mode,speechiness,...,var_segment_loudness_max_end_diff,mean_segment_num_pitches,var_segment_num_pitches,mean_segment_num_pure_pitches,var_segment_num_pure_pitches,mean_segment_timbre,var_segment_timbre,num_tatums,mean_tatum_duration,var_tatum_duration
0,All I Want For Christmas Is You,241106.0,69.0,81.000000,0.336,0.627,7.0,-7.463,1.0,0.0384,...,45.33173,12.0,0.0,2.263092,2.966942,1864.665905,1.180701e+06,1106,0.215763,0.00616
1,Rockin' Around The Christmas Tree,126266.0,62.0,59.000000,0.589,0.472,8.0,-8.749,1.0,0.0502,...,45.33173,12.0,0.0,2.263092,2.966942,1864.665905,1.180701e+06,1106,0.215763,0.00616
2,Jingle Bell Rock,130973.0,62.0,50.000000,0.754,0.424,2.0,-8.463,1.0,0.0363,...,45.33173,12.0,0.0,2.263092,2.966942,1864.665905,1.180701e+06,1106,0.215763,0.00616
3,A Holly Jolly Christmas,135533.0,54.0,48.000000,0.683,0.375,0.0,-13.056,1.0,0.0303,...,45.33173,12.0,0.0,2.263092,2.966942,1864.665905,1.180701e+06,1106,0.215763,0.00616
4,Circles,215280.0,86.0,91.000000,0.695,0.762,0.0,-3.497,1.0,0.0395,...,45.33173,12.0,0.0,2.263092,2.966942,1864.665905,1.180701e+06,1106,0.215763,0.00616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,Christmas Isn't Canceled (Just You),231549.0,41.0,77.000000,0.580,0.789,1.0,-4.918,0.0,0.0457,...,45.33173,12.0,0.0,2.263092,2.966942,1864.665905,1.180701e+06,1106,0.215763,0.00616
1419,Moved To Miami,222225.0,66.0,88.500000,0.717,0.444,1.0,-11.126,1.0,0.1800,...,45.33173,12.0,0.0,2.263092,2.966942,1864.665905,1.180701e+06,1106,0.215763,0.00616
1420,Hibachi,170413.0,69.0,86.333333,0.681,0.522,5.0,-8.740,0.0,0.3030,...,45.33173,12.0,0.0,2.263092,2.966942,1864.665905,1.180701e+06,1106,0.215763,0.00616
1421,Thailand,200958.0,70.0,84.000000,0.875,0.478,7.0,-10.562,1.0,0.2180,...,45.33173,12.0,0.0,2.263092,2.966942,1864.665905,1.180701e+06,1106,0.215763,0.00616


### Testing Different Popularity Metrics

In [50]:
def popularity_metrics(df, score_type='basic'):
    metrics = ['peak_rank', 'debut_rank', 'lifetime_peak_rank', 'sensationality', 'avg_rank_score', 'std_rank_score', 'time_on_chart', 'num_occurrences', 'rank_sum', 'skewness', 'kurtosis', 'spotify_popularity', 'top_10', 'top_20', 'top_50']
    metric_vals = []

    for i in range(len(df)):
        song = df.iloc[i]
        billboard_entries = billboard_df[billboard_df['title'] == song['billboard_name']].sort_values(by='date')
        value_counts = billboard_entries['rank'].value_counts()
        ranks = value_counts.index
        rank_counts = value_counts.values

        # max rank
        peak = min(ranks)
        if score_type == 'classic':
            ranks = [rank_score_classic(peak, ra) for ra in ranks]
            scaled = False
        elif score_type == 'score_01':
            ranks = [rank_score_01(peak, ra) for ra in ranks]
            scaled = True
        elif score_type == 'score_02':
            ranks = [rank_score_02(peak, ra) for ra in ranks]
            scaled = True
        else:
            ranks = [rank_score_basic(ra) for ra in ranks]
            scaled = True

        # sensationality
        sensation = squiggle(rank_counts, ranks, scaled=scaled)

        # mean rank
        avg_rank = np.mean(ranks)

        # rank std
        std_rank = np.std(ranks)

        # length
        time_on_chart = np.max(billboard_entries['weeks'].values)

        # lifetime_peak
        lifetime_peak = np.min(billboard_entries['peakPos'].values)

        # debut rank
        debut_rank = billboard_entries['rank'].iloc[0]

        # number of occurrences
        num_occurrences = len(billboard_entries)

        rank_sum = sum(ranks)

        skewness = skew(ranks)

        kurt = kurtosis(ranks)

        top_10 = any(ele <= 10 for ele in billboard_entries['rank'])
        top_20 = any(ele <= 20 for ele in billboard_entries['rank'])
        top_50 = any(ele <= 50 for ele in billboard_entries['rank'])

        metric_vals.append([peak, debut_rank, lifetime_peak, sensation, avg_rank, std_rank, time_on_chart, num_occurrences, rank_sum, skewness, kurt, song['spotify_popularity'], top_10, top_20, top_50])

    return pd.DataFrame(data=metric_vals, columns=metrics)

In [51]:
pop_metrics_basic = popularity_metrics(songs_df, 'basic')
pop_metrics_basic

,peak_rank,debut_rank,lifetime_peak_rank,sensationality,avg_rank_score,std_rank_score,time_on_chart,num_occurrences,rank_sum,skewness,kurtosis,spotify_popularity,top_10,top_20,top_50
0,1,1,1,0.999999,0.270183,0.316913,50,14,2.161467,1.395650,0.673814,69.0,True,True,True
1,2,2,2,0.998958,0.123908,0.156764,44,17,1.362990,1.352697,0.450922,62.0,True,True,True
2,3,3,3,0.981122,0.151254,0.099182,41,12,1.210036,0.477332,-0.985764,62.0,True,True,True
3,4,4,4,0.931350,0.121060,0.075093,25,11,0.968480,0.378995,-1.307941,54.0,True,True,True
4,1,5,1,0.999999,0.153094,0.215097,61,47,2.908793,3.182407,9.711951,86.0,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,79,79,79,0.012658,0.012658,0.000000,1,1,0.012658,0.000000,-3.000000,41.0,False,False,False
1419,85,85,85,0.011764,0.011765,0.000000,1,1,0.011765,0.000000,-3.000000,66.0,False,False,False
1420,91,91,91,0.010989,0.010989,0.000000,1,1,0.010989,0.000000,-3.000000,69.0,False,False,False
1421,95,95,95,0.010526,0.010526,0.000000,1,1,0.010526,0.000000,-3.000000,70.0,False,False,False


In [52]:
data_df = songs_df.copy()
pm_df = pop_metrics_basic.copy()
data_df.drop(labels=['spotify_popularity', 'spotify_artist_popularity_mean'], axis=1, inplace=True)

song_audio_analysis_files = data_df.pop('audio_analysis_file')

data_df.to_csv("../data/data_app.csv")
pm_df.to_csv("../data/popularity_metrics_app.csv")

song_billboard_names = data_df.pop('billboard_name')

pm_df['debut_rank'] = pm_df['debut_rank'].apply(lambda x: 1/x)
pm_df['avg_rank_score'] = pm_df['avg_rank_score'].apply(lambda x: round(x, 2))

data_df[data_df.columns] = StandardScaler().fit_transform(data_df)
pm_df[pm_df.columns] = StandardScaler().fit_transform(pm_df)

# data_df = data_df.filter(regex='^((?!mfcc).)*$')

data_df.to_csv("../data/data.csv")
pm_df.to_csv("../data/popularity_metrics.csv")

# Reducing Initial Feature Set
To reduce the number of features used going forward, we estimate mutual info for each variable in relation to the chosen predictor variable

In [53]:
from IPython.display import display
def mutual_information_feature_reduction(X, y, mi_type='regression', threshold=0.0, display_mi=False):
    assert mi_type == "regression" or mi_type == "classification", "valid mi_type value are: 'regression' or 'classification'"
    if mi_type == 'regression':
        mi = mutual_info_regression(X, y, random_state=random_state)
    else:
        mi = mutual_info_classif(X, y, random_state=random_state)
    if display_mi:
        display(pd.Series(mi, index=X.columns).sort_values(ascending=False).to_frame().style.bar(color='red'))
    return X.loc[:, mi > threshold]

In [54]:
predictors = []
for dep in pop_metrics_basic.columns:
    reduced = mutual_information_feature_reduction(data_df, pop_metrics_basic[dep], mi_type='regression', display_mi=True, threshold=0.005)
    predictors.append(list(reduced.columns))
predictors_df = pd.DataFrame(data=[predictors], columns=pop_metrics_basic.columns)
predictors_df.to_csv("./regression_features.csv", index=False)

,0
zero_crossing_rate_mean,0.051238
speechiness,0.033464
mean_segment_timbre,0.033415
mfcc7_var,0.031092
mfcc4_mean,0.030547
mfcc5_mean,0.028091
length,0.027519
valence,0.026533
mean_bar_duration,0.026510
energy,0.026353


,0
mfcc13_mean,0.041001
mfcc16_mean,0.040592
zero_crossing_rate_mean,0.034482
var_segment_loudness_end,0.034259
mfcc19_var,0.028271
energy,0.027828
acousticness,0.027572
spectral_bandwidth_var,0.026750
mfcc12_var,0.023719
explicit_False,0.023675


,0
zero_crossing_rate_mean,0.040483
mean_bar_duration,0.036972
harmony_var,0.035532
valence,0.035410
mfcc6_mean,0.030772
mfcc1_var,0.029020
mean_segment_timbre,0.027236
length,0.026967
mean_section_loudness,0.026097
mfcc5_mean,0.025933


,0
rms_var,0.058356
mfcc8_var,0.055044
mfcc6_var,0.046919
mfcc7_var,0.044353
liveness,0.042916
spectral_bandwidth_var,0.037353
mfcc4_mean,0.036317
mfcc10_var,0.032623
explicit_True,0.031706
mfcc3_var,0.031440


,0
mfcc10_var,0.053835
mfcc19_mean,0.043538
perceptr_var,0.043515
spectral_centroid_mean,0.042980
spectral_bandwidth_mean,0.039750
explicit_True,0.036316
mfcc3_mean,0.035394
mfcc5_mean,0.030805
length,0.030448
harmony_var,0.030299


...

,0
mfcc4_var,0.040939
length,0.040316
chroma_stft_mean,0.040068
var_segment_loudness_start_max_diff,0.038721
spectral_centroid_var,0.036776
mfcc8_var,0.036321
rms_var,0.033741
perceptr_var,0.033678
mfcc11_var,0.032989
var_segment_num_pitches,0.032406


,0
speechiness,0.087085
mfcc19_var,0.087045
liveness,0.068147
mfcc9_var,0.063727
perceptr_var,0.051961
valence,0.049516
rms_var,0.040145
harmony_var,0.037369
mfcc3_var,0.037023
acousticness,0.035763


,0
num_bars,0.056276
mean_section_duration,0.035674
var_tatum_duration,0.026641
mean_segment_duration,0.024914
var_segment_loudness_start_end_diff,0.018289
mfcc9_mean,0.015322
mfcc19_var,0.015223
rolloff_var,0.014328
rolloff_mean,0.014328
num_sections,0.013578


,0
num_bars,0.045143
var_tatum_duration,0.037862
mfcc19_var,0.023820
time_signature,0.020272
mean_segment_num_pure_pitches,0.020056
mean_segment_loudness_start_max_diff,0.020023
instrumentalness,0.018856
mean_segment_duration,0.016491
rms_var,0.015958
mfcc12_var,0.015449


,0
var_tatum_duration,0.052088
length,0.041725
mfcc18_var,0.036112
mfcc14_var,0.032578
mfcc13_var,0.024609
mean_segment_loudness_start_max_diff,0.023901
mfcc12_var,0.023657
mfcc5_mean,0.022788
mfcc19_var,0.021302
zero_crossing_rate_var,0.020800


In [55]:
# data_df['title'] = songs_df['billboard_name']
# bb_df = billboard_df.merge(data_df, how='inner', on='title')
# bb_df['top_50'] = bb_df.apply(lambda x: 1 if x['rank'] >= 50 else 0, axis=1)
# bb_df['top_20'] = bb_df.apply(lambda x: 1 if x['rank'] >= 20 else 0, axis=1)
# bb_df['top_10'] = bb_df.apply(lambda x: 1 if x['rank'] >= 10 else 0, axis=1)
# top_50 = bb_df.pop('top_50')
# top_20 = bb_df.pop('top_20')
# top_10 = bb_df.pop('top_10')
# clf_metrics = pd.concat([top_50, top_20, top_10], axis=1)
# names = bb_df.pop('title')
# bb_df.drop(labels=['peakPos', 'lastPos', 'weeks', 'isNew', 'date', 'rank'], axis=1, inplace=True)

In [56]:
predictors = []
clf_metrics = pop_metrics_basic[['top_10', 'top_20', 'top_50']]
for dep in clf_metrics.columns:
    reduced = mutual_information_feature_reduction(data_df, clf_metrics[dep], mi_type='classification', display_mi=True, threshold=0.005)
    predictors.append(list(reduced.columns))
predictors_df = pd.DataFrame(data=[predictors], columns=clf_metrics.columns)
predictors_df.to_csv("./classification_features.csv", index=False)

,0
mfcc9_mean,0.015322
mfcc19_var,0.015223
rolloff_var,0.014328
rolloff_mean,0.014328
rms_var,0.013132
harmony_mean,0.010898
mfcc3_var,0.010847
mfcc4_mean,0.010752
var_segment_loudness_max_time,0.010454
mfcc13_var,0.009289


,0
mfcc19_var,0.023763
mean_segment_duration,0.021083
mean_beat_duration,0.016364
rms_var,0.015753
mfcc12_var,0.015449
num_segments,0.014395
num_bars,0.014381
mfcc6_mean,0.013048
var_segment_loudness_max_end_diff,0.012529
mfcc13_var,0.010682


,0
length,0.041140
mfcc18_var,0.036112
mfcc14_var,0.032588
var_segment_loudness_max_end_diff,0.027222
mfcc13_var,0.024301
mean_segment_loudness_end,0.024200
var_segment_num_pure_pitches,0.023928
mfcc12_var,0.023657
mfcc5_mean,0.022788
var_section_loudness,0.021730


In [57]:
# reduced_cols = mutual_information_feature_reduction(bb_df, top_20, mi_type='classification', display_mi=False, threshold=0.05)
# reduced_cols

In [58]:
# reduced_cols = mutual_information_feature_reduction(bb_df, top_10, mi_type='classification', display_mi=False, threshold=0.05)
# reduced_cols

In [59]:
# bb_df['top_50'] = top_50
# bb_df['top_20'] = top_20
# bb_df['top_10'] = top_10
# bb_df['title'] = names
#


In [60]:
# pop_metrics_basic = pop_metrics_basic.merge(top_10, how="innner", on=)
complete_df = pd.concat([data_df, pop_metrics_basic], axis=1)
complete_df['billboard_name'] = song_billboard_names

print(data_df.shape)
print(pop_metrics_basic.shape)
complete_df.to_csv("../data/complete.csv")
# com)

(1423, 111)
(1423, 15)


In [61]:
data_df

,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,var_segment_loudness_max_end_diff,mean_segment_num_pitches,var_segment_num_pitches,mean_segment_num_pure_pitches,var_segment_num_pure_pitches,mean_segment_timbre,var_segment_timbre,num_tatums,mean_tatum_duration,var_tatum_duration
0,0.796196,-2.178146,0.030618,0.513133,-0.277509,0.778521,-0.806495,-0.236372,-0.153169,-0.839151,...,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0
1,-1.466521,-0.508462,-0.921567,0.790982,-0.776142,0.778521,-0.712676,1.541868,-0.153169,2.511305,...,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0
2,-1.373778,0.580463,-1.216437,-0.876113,-0.665248,0.778521,-0.823191,1.656466,-0.153169,-0.882363,...,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0
3,-1.283932,0.111895,-1.517450,-1.431812,-2.446135,0.778521,-0.870895,1.403561,-0.153169,-0.799026,...,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0
4,0.287341,0.191090,0.859939,-1.431812,1.260264,0.778521,-0.797749,-0.125726,-0.126556,-0.719547,...,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418,0.607892,-0.567858,1.025804,-1.153963,0.709287,-1.284487,-0.748454,-0.714917,-0.153169,-0.258879,...,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0
1419,0.424180,0.336280,-1.093574,-1.153963,-1.697798,0.778521,0.319326,-0.806990,-0.149733,-0.505803,...,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0
1420,-0.596684,0.098696,-0.614410,-0.042566,-0.772652,-1.284487,1.297263,-0.808966,-0.153169,1.816830,...,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0
1421,0.005151,1.379008,-0.884708,0.513133,-1.479113,0.778521,0.621453,-0.856109,-0.153169,-0.251162,...,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,-1.0,1.0


In [62]:
spotify_df[spotify_df['spotify_id'].duplicated()]

KeyError: 'spotify_id'